In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
CSV_NAME = "transactions_p2_2_2_2.csv"

In [4]:
#df = pd.read_csv("transactions_test.csv")
#df

In [5]:
df = pd.read_csv(f"./fourth_div/{CSV_NAME}")
df.head()

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned
0,2021-05-08 21:15:43.572000+00:00,53419793,0,968,927384804,610055757,83301558,140361932,34.99,29.74,9.400,NaN,0.0,TRENDYOLMİLLA&TRENDYOL MAN - Yeni Sezonda Fırs...,15.0,0,True,0.0
1,2021-05-04 18:41:30.548000+00:00,53419793,0,243428,913068910,602013939,101371984,169432491,120.00,39.90,11.330,NaN,0.0,20 TL Üzeri %50 İndirim,50.0,1,False,0.0
2,2021-05-08 21:15:43.572000+00:00,53419793,0,132176,927384806,610055757,75874692,129862946,39.99,29.99,4.700,NaN,0.0,Şimdi Moda - Sepette %25 İndirim,25.0,0,True,0.0
3,2021-05-14 13:56:48.311000+00:00,53419793,0,968,943730365,619044657,87911283,147075951,99.99,50.00,9.400,NaN,0.0,TRENDYOLMİLLA & TRENDYOL MAN - Sepette %50 - S...,50.0,0,False,0.0
4,2021-05-24 09:35:12.414000+00:00,53419793,0,528,967420526,632188079,38924747,68861866,40.00,40.00,5.335,NaN,0.0,NaN,NaN,0,False,0.0


In [6]:
#df.isna().sum()

In [7]:
#df.ship_cost.hist()

In [8]:
#df.ship_cost.mean()

In [9]:
df["ship_cost"] = df["ship_cost"].fillna(5)

In [10]:
#df.columns

In [11]:
#df.info()

In [12]:
df["is_saved_card_trx"] = df["is_saved_card_trx"].astype(int)

In [13]:
df["original_discount_diff"] = df["original_price"] - df["discounted_price"]
df["discount_div_original"] =  df["original_discount_diff"] / df["original_price"]

In [14]:
df["has_coupon"] = df["coupon_id"].notna().astype(int)
df["has_coupon"].value_counts()

0    647495
1     25840
Name: has_coupon, dtype: int64

In [15]:
df["used_promo"] = df["promotion_name"].notna().astype(int)
df["used_promo"].value_counts()

1    342649
0    330686
Name: used_promo, dtype: int64

In [16]:
df.shape

(673335, 22)

In [17]:
revs = pd.read_csv("revs_clean.csv")
#revs

In [18]:
df = df.merge(revs, on=["product_content_id", "supplier_id"], how="left")

In [19]:
df.shape

(673335, 26)

In [20]:
df.columns

Index(['order_date', 'user_id', 'is_elite_user', 'supplier_id',
       'order_line_item_id', 'order_parent_id', 'product_content_id',
       'product_variant_id', 'original_price', 'discounted_price', 'ship_cost',
       'coupon_id', 'coupon_discount', 'promotion_name',
       'promotion_award_value', 'is_wallet_trx', 'is_saved_card_trx',
       'is_returned', 'original_discount_diff', 'discount_div_original',
       'has_coupon', 'used_promo', 'rate_min', 'rate_max', 'rate_median',
       'rate_mean'],
      dtype='object')

In [21]:
returns = pd.read_csv("supplier_defective_return.csv")
#returns

In [22]:
df = df.merge(returns, on="supplier_id")
df.shape

(673330, 27)

In [23]:
group_cols = ["user_id", "product_content_id", "order_parent_id"]

In [24]:
df_grouped = pd.DataFrame(df.groupby(group_cols)["is_returned"].max()).reset_index()
df_grouped

,user_id,product_content_id,order_parent_id,is_returned
0,53419793,38924746,632188079,0.0
1,53419793,38924747,632188079,0.0
2,53419793,74906192,602013939,0.0
3,53419793,75874692,610055757,0.0
4,53419793,75874694,610055757,0.0
...,...,...,...,...
647337,71173209,39088676,722304223,0.0
647338,71173209,74591958,722304223,0.0
647339,71173209,75798504,722304223,0.0
647340,71173209,81125378,722304223,0.0


In [25]:
df.is_returned.value_counts()

0.0    617869
1.0     55461
Name: is_returned, dtype: int64

In [26]:
df_grouped = df_grouped.merge(pd.DataFrame(df[[*group_cols, "is_elite_user"]].drop_duplicates()).reset_index(drop=True), on=group_cols)
df_grouped

,user_id,product_content_id,order_parent_id,is_returned,is_elite_user
0,53419793,38924746,632188079,0.0,0
1,53419793,38924747,632188079,0.0,0
2,53419793,74906192,602013939,0.0,0
3,53419793,75874692,610055757,0.0,0
4,53419793,75874694,610055757,0.0,0
...,...,...,...,...,...
647337,71173209,39088676,722304223,0.0,0
647338,71173209,74591958,722304223,0.0,0
647339,71173209,75798504,722304223,0.0,0
647340,71173209,81125378,722304223,0.0,0


In [27]:
df.is_elite_user.value_counts()

0    611861
1     61469
Name: is_elite_user, dtype: int64

In [28]:
#df.groupby(group_cols)["supplier_id"].nunique().sort_values(ascending=False)

In [29]:
#df_grouped = df_grouped.merge(df.groupby(group_cols)["supplier_id"].unique().reset_index(),
#                                              on=group_cols)
#df_grouped

In [30]:
#df.groupby(group_cols)["order_line_item_id"].nunique().value_counts()

In [31]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["order_line_item_id"].nunique().reset_index(),
                                              on=group_cols)
df_grouped.shape

(647342, 6)

In [32]:
#df_grouped.order_line_item_id.value_counts()

In [33]:
num_items_cart = dict(df.groupby("order_parent_id")["order_line_item_id"].nunique())

In [34]:
df_grouped["num_items_cart"] = df_grouped["order_parent_id"].apply(lambda x: num_items_cart[x])
df_grouped.shape

(647342, 7)

In [35]:
df.groupby(group_cols)["original_price"].nunique().value_counts()

1    646957
2       374
3        11
Name: original_price, dtype: int64

In [36]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["original_price"].unique().reset_index(),
                                              on=group_cols)
df_grouped

,user_id,product_content_id,order_parent_id,is_returned,is_elite_user,order_line_item_id,num_items_cart,original_price
0,53419793,38924746,632188079,0.0,0,1,3,[33.0]
1,53419793,38924747,632188079,0.0,0,1,3,[40.0]
2,53419793,74906192,602013939,0.0,0,1,6,[49.99]
3,53419793,75874692,610055757,0.0,0,1,4,[39.99]
4,53419793,75874694,610055757,0.0,0,1,4,[46.65]
...,...,...,...,...,...,...,...,...
647337,71173209,39088676,722304223,0.0,0,1,5,[99.9]
647338,71173209,74591958,722304223,0.0,0,1,5,[79.95]
647339,71173209,75798504,722304223,0.0,0,1,5,[119.95]
647340,71173209,81125378,722304223,0.0,0,1,5,[59.99]


In [37]:
#df_grouped = df_grouped.drop("is_returned", axis=1)

In [38]:
df_grouped["original_price_max"] = df_grouped["original_price"].apply(lambda x: max(x))
df_grouped["original_price_min"] = df_grouped["original_price"].apply(lambda x: min(x))
df_grouped["original_price_mean"] = df_grouped["original_price"].apply(lambda x: np.mean(x))

In [39]:
df.groupby(group_cols)["discounted_price"].nunique().value_counts()

1    646692
2       636
3        14
Name: discounted_price, dtype: int64

In [40]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["discounted_price"].unique().reset_index(),
                                              on=group_cols)
#df_grouped

In [41]:
df_grouped["discounted_price_max"] = df_grouped["discounted_price"].apply(lambda x: max(x))
df_grouped["discounted_price_min"] = df_grouped["discounted_price"].apply(lambda x: min(x))
df_grouped["discounted_price_mean"] = df_grouped["discounted_price"].apply(lambda x: np.mean(x))

In [42]:
df.groupby(group_cols)["ship_cost"].nunique().value_counts()

1    647008
2       331
3         3
Name: ship_cost, dtype: int64

In [43]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["ship_cost"].unique().reset_index(),
                                              on=group_cols)
#df_grouped

In [44]:
df_grouped["ship_cost_max"] = df_grouped["ship_cost"].apply(lambda x: max(x))
df_grouped["ship_cost_min"] = df_grouped["ship_cost"].apply(lambda x: min(x))
df_grouped["ship_cost_mean"] = df_grouped["ship_cost"].apply(lambda x: np.mean(x))

In [45]:
#df_grouped.isna().sum()

In [46]:
df.groupby(group_cols)["has_coupon"].nunique().value_counts()

1    647313
2        29
Name: has_coupon, dtype: int64

In [47]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["has_coupon"].max().reset_index(),
                                              on=group_cols)
#df_grouped

In [48]:
df_grouped["has_coupon"].value_counts()

0    622096
1     25246
Name: has_coupon, dtype: int64

In [49]:
df.groupby(group_cols)["used_promo"].nunique().value_counts()

1    646695
2       647
Name: used_promo, dtype: int64

In [50]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["used_promo"].max().reset_index(),
                                              on=group_cols)
df_grouped

,user_id,product_content_id,order_parent_id,is_returned,is_elite_user,order_line_item_id,num_items_cart,original_price,original_price_max,original_price_min,...,discounted_price,discounted_price_max,discounted_price_min,discounted_price_mean,ship_cost,ship_cost_max,ship_cost_min,ship_cost_mean,has_coupon,used_promo
0,53419793,38924746,632188079,0.0,0,1,3,[33.0],33.00,33.00,...,[33.0],33.00,33.00,33.00,[5.335],5.335,5.335,5.335,0,0
1,53419793,38924747,632188079,0.0,0,1,3,[40.0],40.00,40.00,...,[40.0],40.00,40.00,40.00,[5.335],5.335,5.335,5.335,0,0
2,53419793,74906192,602013939,0.0,0,1,6,[49.99],49.99,49.99,...,[29.99],29.99,29.99,29.99,[8.88],8.880,8.880,8.880,0,0
3,53419793,75874692,610055757,0.0,0,1,4,[39.99],39.99,39.99,...,[29.99],29.99,29.99,29.99,[4.7],4.700,4.700,4.700,0,1
4,53419793,75874694,610055757,0.0,0,1,4,[46.65],46.65,46.65,...,[34.99],34.99,34.99,34.99,[4.7],4.700,4.700,4.700,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647337,71173209,39088676,722304223,0.0,0,1,5,[99.9],99.90,99.90,...,[53.91],53.91,53.91,53.91,[9.39],9.390,9.390,9.390,0,1
647338,71173209,74591958,722304223,0.0,0,1,5,[79.95],79.95,79.95,...,[31.98],31.98,31.98,31.98,[2.9600000000000004],2.960,2.960,2.960,0,0
647339,71173209,75798504,722304223,0.0,0,1,5,[119.95],119.95,119.95,...,[47.98],47.98,47.98,47.98,[2.9600000000000004],2.960,2.960,2.960,0,0
647340,71173209,81125378,722304223,0.0,0,1,5,[59.99],59.99,59.99,...,[59.99],59.99,59.99,59.99,[8.88],8.880,8.880,8.880,0,0


In [51]:
df.groupby(group_cols)["is_wallet_trx"].nunique().value_counts()

1    647342
Name: is_wallet_trx, dtype: int64

In [52]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["is_wallet_trx"].max().reset_index(),
                                              on=group_cols)
#df_grouped

In [53]:
df.groupby(group_cols)["is_saved_card_trx"].nunique().value_counts()

1    647342
Name: is_saved_card_trx, dtype: int64

In [54]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["is_saved_card_trx"].max().reset_index(),
                                              on=group_cols)
df_grouped

,user_id,product_content_id,order_parent_id,is_returned,is_elite_user,order_line_item_id,num_items_cart,original_price,original_price_max,original_price_min,...,discounted_price_min,discounted_price_mean,ship_cost,ship_cost_max,ship_cost_min,ship_cost_mean,has_coupon,used_promo,is_wallet_trx,is_saved_card_trx
0,53419793,38924746,632188079,0.0,0,1,3,[33.0],33.00,33.00,...,33.00,33.00,[5.335],5.335,5.335,5.335,0,0,0,0
1,53419793,38924747,632188079,0.0,0,1,3,[40.0],40.00,40.00,...,40.00,40.00,[5.335],5.335,5.335,5.335,0,0,0,0
2,53419793,74906192,602013939,0.0,0,1,6,[49.99],49.99,49.99,...,29.99,29.99,[8.88],8.880,8.880,8.880,0,0,1,0
3,53419793,75874692,610055757,0.0,0,1,4,[39.99],39.99,39.99,...,29.99,29.99,[4.7],4.700,4.700,4.700,0,1,0,1
4,53419793,75874694,610055757,0.0,0,1,4,[46.65],46.65,46.65,...,34.99,34.99,[4.7],4.700,4.700,4.700,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647337,71173209,39088676,722304223,0.0,0,1,5,[99.9],99.90,99.90,...,53.91,53.91,[9.39],9.390,9.390,9.390,0,1,0,0
647338,71173209,74591958,722304223,0.0,0,1,5,[79.95],79.95,79.95,...,31.98,31.98,[2.9600000000000004],2.960,2.960,2.960,0,0,0,0
647339,71173209,75798504,722304223,0.0,0,1,5,[119.95],119.95,119.95,...,47.98,47.98,[2.9600000000000004],2.960,2.960,2.960,0,0,0,0
647340,71173209,81125378,722304223,0.0,0,1,5,[59.99],59.99,59.99,...,59.99,59.99,[8.88],8.880,8.880,8.880,0,0,0,0


In [55]:
df_grouped.shape

(647342, 23)

In [56]:
user_dem = pd.read_csv("user_demographics.csv")
user_dem["membership_date"] = pd.to_datetime(user_dem["membership_date"])
user_dem["membership_year"] = user_dem["membership_date"].dt.year
user_dem["gender"] = user_dem["gender"].replace({"Bayan": 1, "Bay": 0, "UNKNOWN": -1, "kvkktalepsilindi": -2})
user_dem = user_dem.drop(["birth_date", "membership_date"], axis=1)
user_dem.head()

,user_id,gender,membership_year
0,22013146,0,2019
1,49707264,0,2020
2,48146878,0,2020
3,53528681,0,2021
4,17523184,0,2018


In [57]:
sum(user_dem["user_id"].duplicated())

0

In [58]:
user_dem.isna().sum()

user_id            0
gender             0
membership_year    0
dtype: int64

In [59]:
df_grouped = df_grouped.merge(user_dem, on="user_id")
df_grouped

,user_id,product_content_id,order_parent_id,is_returned,is_elite_user,order_line_item_id,num_items_cart,original_price,original_price_max,original_price_min,...,ship_cost,ship_cost_max,ship_cost_min,ship_cost_mean,has_coupon,used_promo,is_wallet_trx,is_saved_card_trx,gender,membership_year
0,53419793,38924746,632188079,0.0,0,1,3,[33.0],33.00,33.00,...,[5.335],5.335,5.335,5.335,0,0,0,0,1,2021
1,53419793,38924747,632188079,0.0,0,1,3,[40.0],40.00,40.00,...,[5.335],5.335,5.335,5.335,0,0,0,0,1,2021
2,53419793,74906192,602013939,0.0,0,1,6,[49.99],49.99,49.99,...,[8.88],8.880,8.880,8.880,0,0,1,0,1,2021
3,53419793,75874692,610055757,0.0,0,1,4,[39.99],39.99,39.99,...,[4.7],4.700,4.700,4.700,0,1,0,1,1,2021
4,53419793,75874694,610055757,0.0,0,1,4,[46.65],46.65,46.65,...,[4.7],4.700,4.700,4.700,0,1,0,1,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647337,71173209,39088676,722304223,0.0,0,1,5,[99.9],99.90,99.90,...,[9.39],9.390,9.390,9.390,0,1,0,0,-1,2021
647338,71173209,74591958,722304223,0.0,0,1,5,[79.95],79.95,79.95,...,[2.9600000000000004],2.960,2.960,2.960,0,0,0,0,-1,2021
647339,71173209,75798504,722304223,0.0,0,1,5,[119.95],119.95,119.95,...,[2.9600000000000004],2.960,2.960,2.960,0,0,0,0,-1,2021
647340,71173209,81125378,722304223,0.0,0,1,5,[59.99],59.99,59.99,...,[8.88],8.880,8.880,8.880,0,0,0,0,-1,2021


In [60]:
products = pd.read_csv("products_clean.csv")
#products = products[~products.product_content_id.duplicated()]
products

,product_content_id,brand_id,category_id,color_id,attribute_value
0,49156804,995186,4979,7009.0,12.0
1,44391553,35046,2760,7005.0,2.0
2,96708156,1074023,3098,6996.0,12.0
3,77520098,961534,3914,7009.0,3.0
4,79713829,1021561,1018,6996.0,25.0
...,...,...,...,...,...
705608,36753009,917455,2144,7015.0,-1.0
705609,42349614,16333,2144,7015.0,-1.0
705610,99357114,977589,2144,7015.0,2.0
705611,72116464,1061811,2144,7015.0,28.0


In [61]:
df_grouped = df_grouped.merge(products, on="product_content_id", how="left")
df_grouped

,user_id,product_content_id,order_parent_id,is_returned,is_elite_user,order_line_item_id,num_items_cart,original_price,original_price_max,original_price_min,...,has_coupon,used_promo,is_wallet_trx,is_saved_card_trx,gender,membership_year,brand_id,category_id,color_id,attribute_value
0,53419793,38924746,632188079,0.0,0,1,3,[33.0],33.00,33.00,...,0,0,0,0,1,2021,849,530,7003.0,6.5
1,53419793,38924747,632188079,0.0,0,1,3,[40.0],40.00,40.00,...,0,0,0,0,1,2021,849,530,7015.0,6.5
2,53419793,74906192,602013939,0.0,0,1,6,[49.99],49.99,49.99,...,0,0,1,0,1,2021,981354,604,7009.0,3.0
3,53419793,75874692,610055757,0.0,0,1,4,[39.99],39.99,39.99,...,0,1,0,1,1,2021,982012,530,6999.0,2.0
4,53419793,75874694,610055757,0.0,0,1,4,[46.65],46.65,46.65,...,0,1,0,1,1,2021,982012,530,7009.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647337,71173209,39088676,722304223,0.0,0,1,5,[99.9],99.90,99.90,...,0,1,0,0,-1,2021,37838,3098,7009.0,-1.0
647338,71173209,74591958,722304223,0.0,0,1,5,[79.95],79.95,79.95,...,0,0,0,0,-1,2021,8605,562,7006.0,3.5
647339,71173209,75798504,722304223,0.0,0,1,5,[119.95],119.95,119.95,...,0,0,0,0,-1,2021,8605,562,6998.0,3.5
647340,71173209,81125378,722304223,0.0,0,1,5,[59.99],59.99,59.99,...,0,0,0,0,-1,2021,3268,604,6998.0,3.5


In [62]:
#df_grouped.columns

In [63]:
#df.groupby(group_cols)["original_discount_diff"].nunique().value_counts()

In [64]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["original_discount_diff"].unique().reset_index(),
                                              on=group_cols)
#df_grouped

In [65]:
df_grouped["original_discount_diff_max"] = df_grouped["original_discount_diff"].apply(lambda x: max(x))
df_grouped["original_discount_diff_min"] = df_grouped["original_discount_diff"].apply(lambda x: min(x))
df_grouped["original_discount_diff_mean"] = df_grouped["original_discount_diff"].apply(lambda x: np.mean(x))

In [66]:
#df_grouped.isna().sum()

In [67]:
#df.groupby(group_cols)["discount_div_original"].nunique().value_counts()

In [68]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["discount_div_original"].unique().reset_index(),
                                              on=group_cols)
#df_grouped

In [69]:
df_grouped["discount_div_original_max"] = df_grouped["discount_div_original"].apply(lambda x: max(x))
df_grouped["discount_div_original_min"] = df_grouped["discount_div_original"].apply(lambda x: min(x))
df_grouped["discount_div_original_mean"] = df_grouped["discount_div_original"].apply(lambda x: np.mean(x))

In [70]:
#df.groupby(group_cols)["returnRate"].nunique().value_counts()

In [71]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["returnRate"].unique().reset_index(),
                                              on=group_cols)
#df_grouped

In [72]:
df_grouped["returnRate_max"] = df_grouped["returnRate"].apply(lambda x: max(x))
df_grouped["returnRate_min"] = df_grouped["returnRate"].apply(lambda x: min(x))
df_grouped["returnRate_mean"] = df_grouped["returnRate"].apply(lambda x: np.mean(x))

In [73]:
#df.groupby(group_cols)["rate_min"].nunique().value_counts()

In [74]:
#df.groupby(group_cols)["rate_max"].nunique().value_counts()

In [75]:
#df.groupby(group_cols)["rate_median"].nunique().value_counts()

In [76]:
#df.groupby(group_cols)["rate_mean"].nunique().value_counts()

In [77]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["rate_min"].unique().reset_index(), on=group_cols)
df_grouped = df_grouped.merge(df.groupby(group_cols)["rate_max"].unique().reset_index(), on=group_cols)
df_grouped = df_grouped.merge(df.groupby(group_cols)["rate_median"].unique().reset_index(), on=group_cols)
df_grouped = df_grouped.merge(df.groupby(group_cols)["rate_mean"].unique().reset_index(), on=group_cols)

In [78]:
df_grouped["rate_min_max"] = df_grouped["rate_min"].apply(lambda x: max(x))
df_grouped["rate_min_min"] = df_grouped["rate_min"].apply(lambda x: min(x))
df_grouped["rate_min_mean"] = df_grouped["rate_min"].apply(lambda x: np.mean(x))

In [79]:
df_grouped["rate_max_max"] = df_grouped["rate_max"].apply(lambda x: max(x))
df_grouped["rate_max_min"] = df_grouped["rate_max"].apply(lambda x: min(x))
df_grouped["rate_max_mean"] = df_grouped["rate_max"].apply(lambda x: np.mean(x))

In [80]:
df_grouped["rate_median_max"] = df_grouped["rate_median"].apply(lambda x: max(x))
df_grouped["rate_median_min"] = df_grouped["rate_median"].apply(lambda x: min(x))
df_grouped["rate_median_mean"] = df_grouped["rate_median"].apply(lambda x: np.mean(x))

In [81]:
df_grouped["rate_mean_max"] = df_grouped["rate_mean"].apply(lambda x: max(x))
df_grouped["rate_mean_min"] = df_grouped["rate_mean"].apply(lambda x: min(x))
df_grouped["rate_mean_mean"] = df_grouped["rate_mean"].apply(lambda x: np.mean(x))

In [82]:
#df.groupby(group_cols)["coupon_discount"].nunique().value_counts()

In [83]:
df_grouped = df_grouped.merge(df.groupby(group_cols)["coupon_discount"].unique().reset_index(), on=group_cols)

In [84]:
df_grouped["coupon_discount_max"] = df_grouped["coupon_discount"].apply(lambda x: max(x))
df_grouped["coupon_discount_min"] = df_grouped["coupon_discount"].apply(lambda x: min(x))
df_grouped["coupon_discount_mean"] = df_grouped["coupon_discount"].apply(lambda x: np.mean(x))

In [85]:
sum(df_grouped.isna().sum() > 0)

12

In [86]:
df_grouped.shape

(647342, 61)

In [87]:
drop_str_cols = list(df_grouped.select_dtypes(object).columns)
drop_str_cols

['original_price',
 'discounted_price',
 'ship_cost',
 'original_discount_diff',
 'discount_div_original',
 'returnRate',
 'rate_min',
 'rate_max',
 'rate_median',
 'rate_mean',
 'coupon_discount']

In [88]:
drop_str_cols = list(df_grouped.select_dtypes(object).columns)
drop_str_cols

['original_price',
 'discounted_price',
 'ship_cost',
 'original_discount_diff',
 'discount_div_original',
 'returnRate',
 'rate_min',
 'rate_max',
 'rate_median',
 'rate_mean',
 'coupon_discount']

In [89]:
df_grouped = df_grouped.drop(drop_str_cols, axis=1)
df_grouped

,user_id,product_content_id,order_parent_id,is_returned,is_elite_user,order_line_item_id,num_items_cart,original_price_max,original_price_min,original_price_mean,...,rate_max_mean,rate_median_max,rate_median_min,rate_median_mean,rate_mean_max,rate_mean_min,rate_mean_mean,coupon_discount_max,coupon_discount_min,coupon_discount_mean
0,53419793,38924746,632188079,0.0,0,1,3,33.00,33.00,33.00,...,5.0,5.0,5.0,5.0,4.642857,4.642857,4.642857,0.0,0.0,0.0
1,53419793,38924747,632188079,0.0,0,1,3,40.00,40.00,40.00,...,5.0,5.0,5.0,5.0,4.477273,4.477273,4.477273,0.0,0.0,0.0
2,53419793,74906192,602013939,0.0,0,1,6,49.99,49.99,49.99,...,5.0,5.0,5.0,5.0,4.517413,4.517413,4.517413,0.0,0.0,0.0
3,53419793,75874692,610055757,0.0,0,1,4,39.99,39.99,39.99,...,5.0,5.0,5.0,5.0,4.045070,4.045070,4.045070,0.0,0.0,0.0
4,53419793,75874694,610055757,0.0,0,1,4,46.65,46.65,46.65,...,5.0,4.0,4.0,4.0,3.784211,3.784211,3.784211,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647337,71173209,39088676,722304223,0.0,0,1,5,99.90,99.90,99.90,...,5.0,4.0,4.0,4.0,3.469388,3.469388,3.469388,0.0,0.0,0.0
647338,71173209,74591958,722304223,0.0,0,1,5,79.95,79.95,79.95,...,5.0,5.0,5.0,5.0,4.557692,4.557692,4.557692,0.0,0.0,0.0
647339,71173209,75798504,722304223,0.0,0,1,5,119.95,119.95,119.95,...,5.0,5.0,5.0,5.0,4.316456,4.316456,4.316456,0.0,0.0,0.0
647340,71173209,81125378,722304223,0.0,0,1,5,59.99,59.99,59.99,...,5.0,5.0,5.0,5.0,4.657854,4.657854,4.657854,0.0,0.0,0.0


In [90]:
sum(df_grouped[group_cols].duplicated())

0

In [91]:
#df_grouped.to_csv("test_feats.csv", index=False)

In [92]:
df_grouped.to_csv(f"./grouped_data/{CSV_NAME}", index=False)